### Consolidando dados num arquivo xlsx para análise no Tableau ###


In [5]:
import os

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/falando_nela_v2/src')
    print("Current working directory (Colab):", os.getcwd())
else:
  print("Current working directory (not Colab):", os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory (Colab): /content/drive/MyDrive/falando_nela_v2/src


In [15]:
import os
import pandas as pd

# 📁 Caminho base dos arquivos de entrada e saída
data_path = "/content/drive/MyDrive/falando_nela_v2/data"
saida_path = os.path.join(data_path, "tableau_por_ano")
os.makedirs(saida_path, exist_ok=True)

# 📘 Arquivos base
arquivo_discursos = os.path.join(data_path, "dados_para_tableau.xlsx")
arquivo_topicos = os.path.join(data_path, "dados_para_tableau_4.xlsx")
arquivo_argumentacao_ano = os.path.join(data_path, "dados_para_tableau_3.xlsx")

# 📗 Lê discursos e senadores
df_discursos = pd.read_excel(arquivo_discursos, sheet_name="Discursos")
df_senadores = pd.read_excel(arquivo_discursos, sheet_name="Senadores")

# Padroniza datas
for col in df_discursos.columns:
    if "Data" in col:
        df_discursos[col] = pd.to_datetime(df_discursos[col], errors="coerce")

# 📙 Lê tópicos de resumo por aba
def carregar_topicos_resumo(filepath):
    xls = pd.ExcelFile(filepath)
    dataframes = {}
    for aba in xls.sheet_names:
        if aba.startswith("TextoResumo_"):
            ano = int(aba.split("_")[1])
            df = pd.read_excel(xls, sheet_name=aba)
            df = df.rename(columns={"Representation": "TopicoResumo"})
            dataframes[ano] = df[["CodigoPronunciamento", "TopicoResumo"]].dropna()
    return dataframes

# 📒 Lê argumentação por aba
def carregar_argumentacao_ano(filepath):
    xls = pd.ExcelFile(filepath)
    dataframes = {}
    for aba in xls.sheet_names:
        if "_" not in aba:
            continue
        tipo, ano = aba.rsplit("_", 1)
        ano = int(ano)
        df = pd.read_excel(xls, sheet_name=aba)
        df["Tipo_Anual"] = tipo
        if ano not in dataframes:
            dataframes[ano] = []
        df = df.rename(columns={
            "Trecho": "Trecho_Anual",
            "Topico": "Topico_Anual",
            "Name": "Topico_Anual",
            "TopicoPredominante": "Topico_Anual",
            "Document": "Trecho_Anual"
        })
        dataframes[ano].append(df)
    return {ano: pd.concat(partes, ignore_index=True) for ano, partes in dataframes.items()}

# Carregar todos os tópicos e argumentos organizados por ano
topicos_por_ano = carregar_topicos_resumo(arquivo_topicos)
argumentacao_por_ano = carregar_argumentacao_ano(arquivo_argumentacao_ano)

# Lista de anos disponíveis
anos = sorted(set(topicos_por_ano.keys()) | set(argumentacao_por_ano.keys()))

# ✨ Gera um arquivo por ano
for ano in anos:
    print(f"📦 Gerando arquivo para {ano}...")

    df_disc_ano = df_discursos[df_discursos["DataPronunciamento"].dt.year == ano]
    df_topicos = topicos_por_ano.get(ano, pd.DataFrame())
    df_arg_ano = argumentacao_por_ano.get(ano, pd.DataFrame())

    # Define nome do arquivo
    nome_arquivo = f"tabela_para_tableau_{ano}.xlsx"
    caminho_saida = os.path.join(saida_path, nome_arquivo)

    # Exporta
    with pd.ExcelWriter(caminho_saida, engine="xlsxwriter") as writer:
        df_disc_ano.to_excel(writer, sheet_name="Discursos", index=False)
        df_topicos.to_excel(writer, sheet_name="TopicosResumo", index=False)
        df_arg_ano.to_excel(writer, sheet_name="Argumentacao", index=False)
        df_senadores.to_excel(writer, sheet_name="Senadores", index=False)

    print(f"✅ Arquivo salvo: {caminho_saida}")


<ipython-input-15-65fcb14f42f6>:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_discursos[col] = pd.to_datetime(df_discursos[col], errors="coerce")
<ipython-input-15-65fcb14f42f6>:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_discursos[col] = pd.to_datetime(df_discursos[col], errors="coerce")


📦 Gerando arquivo para 2007...
✅ Arquivo salvo: /content/drive/MyDrive/falando_nela_v2/data/tableau_por_ano/tabela_para_tableau_2007.xlsx
📦 Gerando arquivo para 2008...
✅ Arquivo salvo: /content/drive/MyDrive/falando_nela_v2/data/tableau_por_ano/tabela_para_tableau_2008.xlsx
📦 Gerando arquivo para 2009...
✅ Arquivo salvo: /content/drive/MyDrive/falando_nela_v2/data/tableau_por_ano/tabela_para_tableau_2009.xlsx
📦 Gerando arquivo para 2010...
✅ Arquivo salvo: /content/drive/MyDrive/falando_nela_v2/data/tableau_por_ano/tabela_para_tableau_2010.xlsx
📦 Gerando arquivo para 2011...
✅ Arquivo salvo: /content/drive/MyDrive/falando_nela_v2/data/tableau_por_ano/tabela_para_tableau_2011.xlsx
📦 Gerando arquivo para 2012...
✅ Arquivo salvo: /content/drive/MyDrive/falando_nela_v2/data/tableau_por_ano/tabela_para_tableau_2012.xlsx
📦 Gerando arquivo para 2013...
✅ Arquivo salvo: /content/drive/MyDrive/falando_nela_v2/data/tableau_por_ano/tabela_para_tableau_2013.xlsx
📦 Gerando arquivo para 2014...
✅ A

In [16]:
import os
import pandas as pd

# 📁 Caminho atualizado para a pasta onde estão os arquivos por ano
folder_path = "/content/drive/MyDrive/falando_nela_v2/data/tableau_por_ano"

# 🔁 Renomeia arquivos que contenham "_sample" no nome
arquivos = [f for f in os.listdir(folder_path) if f.endswith(".xlsx") and "_sample" in f]
for nome_antigo in arquivos:
    nome_novo = nome_antigo.replace("_sample", "")
    os.rename(os.path.join(folder_path, nome_antigo), os.path.join(folder_path, nome_novo))

# 📄 Lista os arquivos relevantes
arquivos_limpos = [f for f in os.listdir(folder_path) if f.startswith("tabela_para_tableau_") and f.endswith(".xlsx")]

# 🔄 Abre os arquivos e acumula dados
discursos_total = []
argumentos_total = []
topicos_total = []

for arquivo in arquivos_limpos:
    path = os.path.join(folder_path, arquivo)
    try:
        xls = pd.ExcelFile(path)

        if "Discursos" in xls.sheet_names:
            df_disc = pd.read_excel(xls, sheet_name="Discursos")
            discursos_total.append(df_disc)

        if "Argumentacao" in xls.sheet_names:
            df_arg = pd.read_excel(xls, sheet_name="Argumentacao")
            argumentos_total.append(df_arg)

        if "TopicosResumo" in xls.sheet_names:
            df_top = pd.read_excel(xls, sheet_name="TopicosResumo")
            topicos_total.append(df_top)

    except Exception as e:
        print(f"⚠️ Erro ao processar {arquivo}: {e}")

# 📊 Concatena os dados
df_disc_total = pd.concat(discursos_total, ignore_index=True) if discursos_total else pd.DataFrame()
df_arg_total = pd.concat(argumentos_total, ignore_index=True) if argumentos_total else pd.DataFrame()
df_top_total = pd.concat(topicos_total, ignore_index=True) if topicos_total else pd.DataFrame()

# 📈 Estatísticas
estatisticas = {
    "Total de discursos únicos": df_disc_total["CodigoPronunciamento"].nunique() if "CodigoPronunciamento" in df_disc_total.columns else 0,
    "Total de linhas em Argumentacao": len(df_arg_total),
    "Total de linhas em TopicosResumo": len(df_top_total),
    "Total de anos (discursos)": df_disc_total["DataPronunciamento"].dt.year.nunique() if "DataPronunciamento" in df_disc_total.columns else 0,
    "Células em Discursos": df_disc_total.shape[0] * df_disc_total.shape[1],
    "Células em Argumentacao": df_arg_total.shape[0] * df_arg_total.shape[1],
    "Células em TopicosResumo": df_top_total.shape[0] * df_top_total.shape[1],
}

estatisticas["Total geral de células"] = (
    estatisticas["Células em Discursos"] +
    estatisticas["Células em Argumentacao"] +
    estatisticas["Células em TopicosResumo"]
)

# 📋 Exibe estatísticas
for chave, valor in estatisticas.items():
    print(f"{chave}: {valor:,}")


Total de discursos únicos: 67,349
Total de linhas em Argumentacao: 75,862
Total de linhas em TopicosResumo: 64,964
Total de anos (discursos): 18
Células em Discursos: 3,030,705
Células em Argumentacao: 834,482
Células em TopicosResumo: 129,928
Total geral de células: 3,995,115


In [19]:
import os
import pandas as pd

# 📁 Caminho da pasta com os arquivos por ano
folder_path = "/content/drive/MyDrive/falando_nela_v2/data/tableau_por_ano"
bm25_path = "/content/drive/MyDrive/falando_nela_v2/data/dados_para_tableau_5.xlsx"

# Lista os arquivos relevantes
arquivos = [f for f in os.listdir(folder_path) if f.startswith("tabela_para_tableau_") and f.endswith(".xlsx")]

# Acumula os dados
discursos = []
argumentos = []
topicos = []
senadores = None

for arquivo in arquivos:
    path = os.path.join(folder_path, arquivo)
    try:
        xls = pd.ExcelFile(path)

        if "Discursos" in xls.sheet_names:
            df_disc = pd.read_excel(xls, sheet_name="Discursos")
            discursos.append(df_disc)

        if "Argumentacao" in xls.sheet_names:
            df_arg = pd.read_excel(xls, sheet_name="Argumentacao")
            argumentos.append(df_arg)

        if "TopicosResumo" in xls.sheet_names:
            df_top = pd.read_excel(xls, sheet_name="TopicosResumo")
            topicos.append(df_top)

        if "Senadores" in xls.sheet_names and senadores is None:
            senadores = pd.read_excel(xls, sheet_name="Senadores")

    except Exception as e:
        print(f"⚠️ Erro ao processar {arquivo}: {e}")

# 🔗 Concatena os dados
df_disc_total = pd.concat(discursos, ignore_index=True)
df_arg_total = pd.concat(argumentos, ignore_index=True)
df_top_total = pd.concat(topicos, ignore_index=True)

# 🔍 Identifica discursos com "constituição"
filtros_const = df_disc_total["TextoIntegral"].str.contains("constituição", case=False, na=False)
codigos_const = df_disc_total.loc[filtros_const, "CodigoPronunciamento"].unique()

# 🧹 Mantém todos os discursos na aba principal
df_disc_final = df_disc_total.copy()

# ✅ Renomeia colunas BM25 e TFIDF como CorpusTodo
df_disc_final = df_disc_final.rename(columns=lambda col: col.replace("BM25_", "BM25_CorpusTodo_").replace("TFIDF_", "TFIDF_CorpusTodo_"))

# 📆 Ordena os DataFrames para melhor leitura
df_disc_final = df_disc_final.sort_values(by="CodigoPronunciamento")

# 🔢 Filtra apenas os argumentos e tópicos dos discursos relevantes, removendo NaNs e colunas vazias
df_arg_final = df_arg_total[df_arg_total["CodigoPronunciamento"].isin(codigos_const)].dropna(how="all").dropna(axis=1, how="all")
df_arg_final = df_arg_final.sort_values(by=["CodigoPronunciamento", "Tipo_Anual"])

df_top_final = df_top_total[df_top_total["CodigoPronunciamento"].isin(codigos_const)].dropna(how="all").dropna(axis=1, how="all")
df_top_final = df_top_final.sort_values(by="CodigoPronunciamento")

# 🔢 Carrega a planilha de BM25 anual
try:
    df_bm25_anual = pd.read_excel(bm25_path, sheet_name=0)
    df_bm25_anual = df_bm25_anual[df_bm25_anual["CodigoPronunciamento"].isin(codigos_const)]
    df_bm25_anual = df_bm25_anual.dropna(how="all").dropna(axis=1, how="all")
    df_bm25_anual = df_bm25_anual.sort_values(by="CodigoPronunciamento")
except Exception as e:
    print(f"⚠️ Erro ao carregar BM25 Anual: {e}")
    df_bm25_anual = pd.DataFrame()

# ✅ Exporta tudo em um único .xlsx na pasta data
output_path = "/content/drive/MyDrive/falando_nela_v2/data/tabela_final.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    df_disc_final.to_excel(writer, sheet_name="Discursos", index=False)
    df_arg_final.to_excel(writer, sheet_name="Argumentacao", index=False)
    df_top_final.to_excel(writer, sheet_name="TopicosResumo", index=False)
    if not df_bm25_anual.empty:
        df_bm25_anual.to_excel(writer, sheet_name="BM25Anual", index=False)
    senadores.to_excel(writer, sheet_name="Senadores", index=False)

print("✅ Arquivo consolidado salvo em:", output_path)


✅ Arquivo consolidado salvo em: /content/drive/MyDrive/falando_nela_v2/data/tabela_final.xlsx


In [20]:
import pandas as pd

# 📁 Caminho para o arquivo final
arquivo_final = "/content/drive/MyDrive/falando_nela_v2/data/tabela_final.xlsx"

# 🔍 Lê todas as abas
xls = pd.ExcelFile(arquivo_final)
df_disc = pd.read_excel(xls, sheet_name="Discursos")
df_arg = pd.read_excel(xls, sheet_name="Argumentacao")
df_top = pd.read_excel(xls, sheet_name="TopicosResumo")
df_bm25_anual = pd.read_excel(xls, sheet_name="BM25Anual")
df_senadores = pd.read_excel(xls, sheet_name="Senadores")

# 🧮 Estatísticas
estatisticas = {
    "Total de discursos (todos)": df_disc["CodigoPronunciamento"].nunique(),
    "Total de discursos analisados (constituição)": df_arg["CodigoPronunciamento"].nunique(),
    "Total de linhas em Argumentacao": len(df_arg),
    "Total de linhas em TopicosResumo": len(df_top),
    "Total de linhas em BM25Anual": len(df_bm25_anual),
    "Total de senadores distintos": df_senadores["CodigoParlamentar"].nunique() if "CodigoParlamentar" in df_senadores.columns else "n/d",
    "Total de anos cobertos": df_disc["DataPronunciamento"].dt.year.nunique() if "DataPronunciamento" in df_disc.columns else "n/d",
    "Células Discursos": df_disc.shape[0] * df_disc.shape[1],
    "Células Argumentacao": df_arg.shape[0] * df_arg.shape[1],
    "Células TopicosResumo": df_top.shape[0] * df_top.shape[1],
    "Células BM25Anual": df_bm25_anual.shape[0] * df_bm25_anual.shape[1],
}

estatisticas["Células totais"] = (
    estatisticas["Células Discursos"] +
    estatisticas["Células Argumentacao"] +
    estatisticas["Células TopicosResumo"] +
    estatisticas["Células BM25Anual"]
)

# 📋 Exibir resultados
for chave, valor in estatisticas.items():
    print(f"{chave}: {valor:,}")


Total de discursos (todos): 67,349
Total de discursos analisados (constituição): 12,621
Total de linhas em Argumentacao: 75,606
Total de linhas em TopicosResumo: 12,601
Total de linhas em BM25Anual: 12,701
Total de senadores distintos: 353
Total de anos cobertos: 18
Células Discursos: 3,030,705
Células Argumentacao: 831,666
Células TopicosResumo: 25,202
Células BM25Anual: 25,402
Células totais: 3,912,975
